<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 2.4: Sequential Logic
**Prev: [Control Flow](2.3_control_flow.ipynb)**<br>
**Next: [FIR Filter](2.5_exercise.ipynb)**

## Motivation
You can't write any meaningful digital logic without state. You can't write any meaningful digital logic without state. You can't write any meaningful digital logic....

Get it? Because without storing intermediate results, you can't get anywhere.

Ok, that bad joke aside, this module will describe how to express common sequential patterns in Chisel. By the end of the module, you should be able to implement and test a shift register in Chisel.

It's important to emphasize that this section will probably not dramatically impress you. Chisel's power is not in new sequential logic patterns, but in the parameterization of a design. Before we demonstrate that capability, we have to learn what these sequential patterns are. Thus, this section will show you that Chisel can do pretty much what Verilog can do - you just need to learn the Chisel syntax.

## Setup

In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/home/joshua/chisel-bootcamp/source/load-ivy.sc"

In [2]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

---
# Registers
The basic stateful element in Chisel is the register, denoted `Reg`.
A `Reg` holds its output value until the rising edge of its clock, at which time it takes on the value of its input.
By default, every Chisel `Module` has an implicit clock that is used by every register in the design.
This saves you from always specifying the same clock all over your code.

<span style="color:blue">**Example: Using a Register**</span><br>
The following code block implements a module that takes the input, adds 1 to it, and connects it as the input of a register.
*Note: The implicit clock can be overridden for multi-clock designs. See the appendix for an example.*

In [5]:
class RegisterModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(12.W))
    val out = Output(UInt(12.W))
  })
  
  val register = Reg(UInt(12.W))
  register := io.in + 1.U
  io.out := register
}

test(new RegisterModule) { c =>
  for (i <- 0 until 100) {
    c.io.in.poke(i.U)
    c.clock.step(1)
    c.io.out.expect((i + 1).U)
  }
}
println("SUCCESS!!")

Elaborating design...
Done elaborating.
test RegisterModule Success: 0 tests passed in 102 cycles in 0.018179 seconds 5610.96 Hz
SUCCESS!!


defined class RegisterModule

The register is created by calling `Reg(tpe)`, where `tpe` is a variable that encodes the type of register we want.
In this example, `tpe` is a 12-bit `UInt`.

Look at what the tester above is doing.
Between calls to `poke()` and `expect`, there is a call to `step(1)`.
This tells the test harness to tick the clock once, which will cause the register to pass its input to its output.

Calling `step(n)` will tick the clock `n` times.

The astute observer will notice that previous testers testing combinational logic did not call `step()`. This is because calling `poke()` on an input immediately propagates the updated values through combinational logic. Calling `step()` is only needed to update state elements in sequential logic. 

The code block below will show the verilog generated by `RegisterModule`.

Note:
* The module has an input for clock (and reset) that you didn't add- this is the implicit clock
* The variable `register` shows up as `reg [11:0]`, as expected
* There is a block sectioned off by `ifdef Randomize` that initialized the register to some random variable before simulation starts
* `register` is updated on `posedge clock`

In [6]:
println(getVerilog(new RegisterModule))

Elaborating design...
Done elaborating.
module RegisterModule(
  input         clock,
  input         reset,
  input  [11:0] io_in,
  output [11:0] io_out
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
`endif // RANDOMIZE_REG_INIT
  reg [11:0] register; // @[cmd4.sc 7:21]
  assign io_out = register; // @[cmd4.sc 9:10]
  always @(posedge clock) begin
    register <= io_in + 12'h1; // @[cmd4.sc 8:21]
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
`ifndef SYNTHESIS
`ifdef FIRRTL_BEFORE_INITIAL
`FIRRTL_BEFORE_INITIAL
`endif
initial begin
  `ifdef RANDOMIZE
    `ifdef INIT_RANDOM
      `INIT_RANDOM
    `endif
    `ifndef VERILATOR
      `ifdef RANDOMIZE_DELAY
        #`RANDOMIZE_DEL

One important note is that Chisel distinguishes between types (like `UInt`) and hardware nodes (like the literal `2.U`, or the output of `myReg`). While
```scala
val myReg = Reg(UInt(2.W))
```
is legal because a Reg needs a data type as a model,
```scala
val myReg = Reg(2.U)
```
is an error because `2.U` is already a hardware node and can't be used as a model.

<span style="color:blue">**Example: RegNext**</span><br>
Chisel has a convenience register object for registers with simple input connections. The previous `Module` can be shortened to the following `Module`. Notice how we didn't need to specify the register bitwidth this time. It gets inferred from the register's output connection, in this case `io.out`.

In [8]:
class RegNextModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(12.W))
    val out = Output(UInt(12.W))
  })
  
  // register bitwidth is inferred from io.out
  io.out := RegNext(io.in + 1.U)
}
visualize(() => new RegNextModule)

test(new RegNextModule) { c =>
  for (i <- 0 until 100) {
    c.io.in.poke(i.U)
    c.clock.step(1)
    c.io.out.expect((i + 1).U)
  }
}
println("SUCCESS!!")

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<?xml-stylesheet href="styles.css" type="text/css"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.43.0 (0)
 -->
<!-- Title: RegNextModule Pages: 1 -->
 
 
 RegNextModule 
 
 
 cluster_RegNextModule 
 
 RegNextModule 
 
<!-- cluster_RegNextModule_clock -->
 
 cluster_RegNextModule_clock 
 
 clock 
 
<!-- cluster_RegNextModule_reset -->
 
 cluster_RegNextModule_reset 
 
 reset 
 
<!-- cluster_RegNextModule_io_in -->
 
 cluster_RegNextModule_io_in 
 
 io_in 
 
<!-- op_add_3 -->
 
 op_add_3 
 
 
 ● 
 
 add 
 
 ● 
 
 1 
 
<!-- cluster_RegNextModule_io_in->op_add_3 -->
 
 cluster_RegNextModule_io_in->op_add_3:in1 
 
 
 
<!-- cluster_RegNextModule_io_out -->
 
 cluster_RegNextModule_io_out 
 
 io_out 
 
<!-- cluster_RegNextModule__T -->
 
 cluster_RegNextModule__T 
 
 _T 
 
<!-- op_tail_4 -->
 
 op_tail_4 
 
 
 ● 
 
 tail 
 
 ● 
 
 1 
 
<!-- cluster_RegNextModule__T->op_tail_4 -->
 
 cluster_RegNextModule__T->op_tail_4:in1 
 
 
 
<!-- op_add_3->cluster_RegNextModule__T -->
 
 op_add_3:out->cluster_RegNextModule__T 
 
 
 
<!-- struct_cluster_RegNextModule_REG -->
 
 struct_cluster_RegNextModule_REG 
 
 
 ● 
 
 REG 
 
 ● 
 
<!-- struct_cluster_RegNextModule_REG->cluster_RegNextModule_io_out -->
 
 struct_cluster_RegNextModule_REG:out->cluster_RegNextModule_io_out 
 
 
 
<!-- op_tail_4->struct_cluster_RegNextModule_REG -->
 
 op_tail_4:out->struct_cluster_RegNextModule_REG:in

Elaborating design...
Done elaborating.
test RegNextModule Success: 0 tests passed in 102 cycles in 0.016707 seconds 6105.29 Hz
SUCCESS!!


defined class RegNextModule

The Verilog looks almost the same as before, though the register name is generated instead of explicity defined.

In [9]:
println(getVerilog(new RegNextModule))

Elaborating design...
Done elaborating.
module RegNextModule(
  input         clock,
  input         reset,
  input  [11:0] io_in,
  output [11:0] io_out
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
`endif // RANDOMIZE_REG_INIT
  reg [11:0] REG; // @[cmd7.sc 8:20]
  assign io_out = REG; // @[cmd7.sc 8:10]
  always @(posedge clock) begin
    REG <= io_in + 12'h1; // @[cmd7.sc 8:27]
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
`ifndef SYNTHESIS
`ifdef FIRRTL_BEFORE_INITIAL
`FIRRTL_BEFORE_INITIAL
`endif
initial begin
  `ifdef RANDOMIZE
    `ifdef INIT_RANDOM
      `INIT_RANDOM
    `endif
    `ifndef VERILATOR
      `ifdef RANDOMIZE_DELAY
        #`RANDOMIZE_DELAY begin end
   

---
# `RegInit`

The register in `RegisterModule` was initialized to random data for simulation.
Unless otherwised specified, registers do not have a reset value (or a reset).
The way to create a register that resets to a given value is with `RegInit`.

For instance, a 12-bit register initialized to zero can be created with the following.
Both versions below are valid and do the same thing:
```scala
val myReg = RegInit(UInt(12.W), 0.U)
val myReg = RegInit(0.U(12.W))
```

The first version has two arguments.
The first argument is a type node that specified the datatype and its width.
The second argument is a hardware node that specified the reset value, in this case 0.

The second version has one argument.
It is a hardware node that specifies the reset value, but normally `0.U`.

<span style="color:blue">**Example: Initialized Register** </span><br>
The following demonstrates using `RegInit()`, initialized to zero.

In [10]:
class RegInitModule extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(12.W))
    val out = Output(UInt(12.W))
  })
  
  val register = RegInit(0.U(12.W))
  register := io.in + 1.U
  io.out := register
}

visualize(() => new RegInitModule)
println(getVerilog(new RegInitModule))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<?xml-stylesheet href="styles.css" type="text/css"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.43.0 (0)
 -->
<!-- Title: RegInitModule Pages: 1 -->
 
 
 RegInitModule 
 
 
 cluster_RegInitModule 
 
 RegInitModule 
 
<!-- cluster_RegInitModule_clock -->
 
 cluster_RegInitModule_clock 
 
 clock 
 
<!-- cluster_RegInitModule_reset -->
 
 cluster_RegInitModule_reset 
 
 reset 
 
<!-- mux_1523396448 -->
 
 mux_1523396448 
 
 
 0 
 
 a? 
 
 ● 
 
 ● 
 
<!-- cluster_RegInitModule_reset->mux_1523396448 -->
 
 cluster_RegInitModule_reset->mux_1523396448:select 
 
 
 
<!-- cluster_RegInitModule_io_in -->
 
 cluster_RegInitModule_io_in 
 
 io_in 
 
<!-- op_add_5 -->
 
 op_add_5 
 
 
 ● 
 
 add 
 
 ● 
 
 1 
 
<!-- cluster_RegInitModule_io_in->op_add_5 -->
 
 cluster_RegInitModule_io_in->op_add_5:in1 
 
 
 
<!-- cluster_RegInitModule_io_out -->
 
 cluster_RegInitModule_io_out 
 
 io_out 
 
<!-- struct_cluster_RegInitModule_register -->
 
 struct_cluster_RegInitModule_register 
 
 
 ● 
 
 register 
 
 ● 
 
<!-- struct_cluster_RegInitModule_register->cluster_RegInitModule_io_out -->
 
 struct_cluster_RegInitModule_register:out->cluster_RegInitModule_io_out 
 
 
 
<!-- cluster_RegInitModule__T -->
 
 cluster_RegInitModule__T 
 
 _T 
 
<!-- op_tail_6 -->
 
 op_tail_6 
 
 
 ● 
 
 tail 
 
 ● 
 
 1 
 
<!-- cluster_RegInitModule__T->op_tail_6 -->
 
 cluster_RegInitModule__T->op_tail_6:in1 
 
 
 
<!-- op_add_5->cluster_RegInitModule__T -->
 
 op_add_5:out->cluster_RegInitModule__T 
 
 
 
<!-- mux_1523396448->struct_cluster_RegInitModule_register -->
 
 mux_1523396448:out->struct_cluster_RegInitModule_register:in 
 
 
 
<!-- op_tail_6->mux_1523396448 -->
 
 op_tail_6:out->mux_1523396448:in2

Elaborating design...
Done elaborating.
module RegInitModule(
  input         clock,
  input         reset,
  input  [11:0] io_in,
  output [11:0] io_out
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
`endif // RANDOMIZE_REG_INIT
  reg [11:0] register; // @[cmd9.sc 7:25]
  wire [11:0] _T_1 = io_in + 12'h1; // @[cmd9.sc 8:21]
  assign io_out = register; // @[cmd9.sc 9:10]
  always @(posedge clock) begin
    if (reset) begin // @[cmd9.sc 7:25]
      register <= 12'h0; // @[cmd9.sc 7:25]
    end else begin
      register <= _T_1; // @[cmd9.sc 8:12]
    end
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
`ifndef SYNTHESIS
`ifdef FIRRTL_BEFORE_INITIAL
`FIRRTL_BEFORE_INITIAL
`endif
in

defined class RegInitModule

Note that the generated verilog now has a block that checks `if (reset)` to reset the register to 0.
Also note that this is inside the `always @(posedge clock)` block.
Chisel's implicit reset is active high and synchronous.
The register is still initialized to random junk before reset is called.
The `PeekPokeTesters` always call reset before running your test, but you can manually call reset as well using the `reset(n)` function, where reset is high for `n` cycles.

---
# Control Flow
Registers are very similar to wires in terms of control flow.
They have last connect semantics and can be assigned to conditionally with `when`, `elsewhen`, and `otherwise`.

<span style="color:blue">**Example: Register Control Flow**</span><br>
The following example finds the maximum value in a sequence of inputs using conditional register assignments.

In [11]:
class FindMax extends Module {
  val io = IO(new Bundle {
    val in  = Input(UInt(10.W))
    val max = Output(UInt(10.W))
  })

  val max = RegInit(0.U(10.W))
  when (io.in > max) {
    max := io.in
  }
  io.max := max
}
visualize(() => new FindMax)
test(new FindMax) { c =>
    c.io.max.expect(0.U)
    c.io.in.poke(1.U)
    c.clock.step(1)
    c.io.max.expect(1.U)
    c.io.in.poke(3.U)
    c.clock.step(1)
    c.io.max.expect(3.U)
    c.io.in.poke(2.U)
    c.clock.step(1)
    c.io.max.expect(3.U)
    c.io.in.poke(24.U)
    c.clock.step(1)
    c.io.max.expect(24.U)
}
println("SUCCESS!!")

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<?xml-stylesheet href="styles.css" type="text/css"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.43.0 (0)
 -->
<!-- Title: FindMax Pages: 1 -->
 
 
 FindMax 
 
 
 cluster_FindMax 
 
 FindMax 
 
<!-- cluster_FindMax_clock -->
 
 cluster_FindMax_clock 
 
 clock 
 
<!-- cluster_FindMax_reset -->
 
 cluster_FindMax_reset 
 
 reset 
 
<!-- mux_1699047765 -->
 
 mux_1699047765 
 
 
 0 
 
 a? 
 
 ● 
 
 ● 
 
<!-- cluster_FindMax_reset->mux_1699047765 -->
 
 cluster_FindMax_reset->mux_1699047765:select 
 
 
 
<!-- cluster_FindMax_io_in -->
 
 cluster_FindMax_io_in 
 
 io_in 
 
<!-- op_gt_7 -->
 
 op_gt_7 
 
 
 ● 
 
 gt 
 
 ● 
 
 ● 
 
<!-- cluster_FindMax_io_in->op_gt_7 -->
 
 cluster_FindMax_io_in->op_gt_7:in1 
 
 
 
<!-- mux_1302135895 -->
 
 mux_1302135895 
 
 
 ● 
 
 a? 
 
 ● 
 
 ● 
 
<!-- cluster_FindMax_io_in->mux_1302135895 -->
 
 cluster_FindMax_io_in->mux_1302135895:in1 
 
 
 
<!-- cluster_FindMax_io_max -->
 
 cluster_FindMax_io_max 
 
 io_max 
 
<!-- struct_cluster_FindMax_max -->
 
 struct_cluster_FindMax_max 
 
 
 ● 
 
 max 
 
 ● 
 
<!-- struct_cluster_FindMax_max->cluster_FindMax_io_max -->
 
 struct_cluster_FindMax_max:out->cluster_FindMax_io_max 
 
 
 
<!-- struct_cluster_FindMax_max->op_gt_7 -->
 
 struct_cluster_FindMax_max:out->op_gt_7:in2 
 
 
 
<!-- struct_cluster_FindMax_max->mux_1302135895 -->
 
 struct_cluster_FindMax_max:out->mux_1302135895:in2 
 
 
 
<!-- cluster_FindMax__T -->
 
 cluster_FindMax__T 
 
 _T 
 
<!-- cluster_FindMax__T->mux_1302135895 -->
 
 cluster_FindMax__T->mux_1302135895:select 
 
 
 
<!-- op_gt_7->cluster_FindMax__T -->
 
 op_gt_7:out->cluster_FindMax__T 
 
 
 
<!-- mux_1699047765->struct_cluster_FindMax_max -->
 
 mux_1699047765:out->struct_cluster_FindMax_max:in 
 
 
 
<!-- mux_1302135895->mux_1699047765 -->
 
 mux_1302135895:out->mux_1699047765:in2

Elaborating design...
Done elaborating.
test FindMax Success: 0 tests passed in 6 cycles in 0.003198 seconds 1876.41 Hz
SUCCESS!!


defined class FindMax

---
# Other Register Examples

Operations called on a register are performed on the **output** of the register, and the kind of operations depend on the register's type.
That means that you can write
```scala
val reg: UInt = Reg(UInt(4.W))
```
which means the value `reg` is of type `UInt` and you can do things you can normally do with `UInt`s, like `+`, `-`, etc.


You aren't restricted to using `UInt`s with registers, you can use any subclass of the base type `chisel3.Data`. This includes `SInt` for signed integers and a lot of other things.

<span style="color:blue">**Example: Comb Filter**</span><br>
The following example shows a comb filter.

In [12]:
class Comb extends Module {
  val io = IO(new Bundle {
    val in  = Input(SInt(12.W))
    val out = Output(SInt(12.W))
  })

  val delay: SInt = Reg(SInt(12.W))
  delay := io.in
  io.out := io.in - delay
}
println(getVerilog(new Comb))

Elaborating design...
Done elaborating.
module Comb(
  input         clock,
  input         reset,
  input  [11:0] io_in,
  output [11:0] io_out
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
`endif // RANDOMIZE_REG_INIT
  reg [11:0] delay; // @[cmd11.sc 7:24]
  assign io_out = $signed(io_in) - $signed(delay); // @[cmd11.sc 9:19]
  always @(posedge clock) begin
    delay <= io_in; // @[cmd11.sc 8:9]
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
`ifndef SYNTHESIS
`ifdef FIRRTL_BEFORE_INITIAL
`FIRRTL_BEFORE_INITIAL
`endif
initial begin
  `ifdef RANDOMIZE
    `ifdef INIT_RANDOM
      `INIT_RANDOM
    `endif
    `ifndef VERILATOR
      `ifdef RANDOMIZE_DELAY
        #`RANDOMIZE_DE

defined class Comb

---
# Exercises
<span style="color:red">**Exercise: Shift Register**</span><br>
Given your new-found registering knowledge, build a module that implements a shift register for a LFSR. Specifically:
- Each element is a single bit wide.
- Has a 4-bit output signal.
- Takes a single input bit, which is the next value into the shift register.
- Outputs the parallel output of the shift register, with the most significant bit being the last element of the shift register and the least significant bit being the first element of the shift register. `Cat` may come in handy.
- **The output initializes at `b0001`.**
- Shifts each clock cycle (no enable signal).
- **Note in Chisel, subword assignment IS ILLEGAL**; something like `out(0) := in` will not work.

<img src="images/shifter4.svg" alt="shift register figure" style="width: 450px" />

A basic Module skeleton, testvector, and Driver invocation is provided below. The first register has been provided for you.

In [15]:
class MyShiftRegister(val init: Int = 1) extends Module {
  val io = IO(new Bundle {
    val in  = Input(Bool())
    val out = Output(UInt(4.W))
  })

  val state = RegInit(UInt(4.W), init.U)
  val nextState = (state << 1) | io.in
  state := nextStatev
  io.out := state
}

test(new MyShiftRegister()) { c =>
  var state = c.init
  for (i <- 0 until 10) {
    // poke in LSB of i (i % 2)
    c.io.in.poke(((i % 2) != 0).B)
    // update expected state
    state = ((state * 2) + (i % 2)) & 0xf
    c.clock.step(1)
    c.io.out.expect(state.U)
  }
}
println("SUCCESS!!")

cmd15.sc:9: not found: value nextStatev
  state := nextStatev
           ^Compilation Failed

: 

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  val nextState = (state << 1) | io.in
  state := nextState
  io.out := state
</pre></article></div></section></div>

<span style="color:red">**Exercise: Parameterized Shift Register (Optional)**</span><br>
Write a shift register that is parameterized by its delay (`n`), its initial value (`init`), and also has an enable input signal (`en`).

In [16]:
// n is the output width (number of delays - 1)
// init state to init
class MyOptionalShiftRegister(val n: Int, val init: BigInt = 1) extends Module {
  val io = IO(new Bundle {
    val en  = Input(Bool())
    val in  = Input(Bool())
    val out = Output(UInt(n.W))
  })

  val state = RegInit(init.U(n.W))
  val nextState = (state << 1) | io.in 
  when(io.en) {
    state := nextState
  }
  io.out := state
}

// test different depths
for (i <- Seq(3, 4, 8, 24, 65)) {
  println(s"Testing n=$i")
  test(new MyOptionalShiftRegister(n = i)) { c =>
    val inSeq = Seq(0, 1, 1, 1, 0, 1, 1, 0, 0, 1)
    var state = c.init
    var i = 0
    c.io.en.poke(true.B)
    while (i < 10 * c.n) {
      // poke in repeated inSeq
      val toPoke = inSeq(i % inSeq.length)
      c.io.in.poke((toPoke != 0).B)
      // update expected state
      state = ((state * 2) + toPoke) & BigInt("1"*c.n, 2)
      c.clock.step(1)
      c.io.out.expect(state.U)

      i += 1
    }
  }
}
println("SUCCESS!!")

Testing n=3
Elaborating design...
Done elaborating.
test MyOptionalShiftRegister Success: 0 tests passed in 32 cycles in 0.006746 seconds 4743.52 Hz
Testing n=4
Elaborating design...
Done elaborating.
test MyOptionalShiftRegister Success: 0 tests passed in 42 cycles in 0.005881 seconds 7141.13 Hz
Testing n=8
Elaborating design...
Done elaborating.
test MyOptionalShiftRegister Success: 0 tests passed in 82 cycles in 0.010818 seconds 7579.98 Hz
Testing n=24
Elaborating design...
Done elaborating.
test MyOptionalShiftRegister Success: 0 tests passed in 242 cycles in 0.036551 seconds 6620.89 Hz
Testing n=65
Elaborating design...
Done elaborating.
test MyOptionalShiftRegister Success: 0 tests passed in 652 cycles in 0.063169 seconds 10321.52 Hz
SUCCESS!!


defined class MyOptionalShiftRegister

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-2" />
<label for="check-2"><strong>Solution</strong></label>
<article>
<pre style="background-color:#f7f7f7">
  val nextState = (state << 1) | io.in
  when (io.en) {
    state  := nextState
  }
  io.out := state
</pre></article></div></section></div>

---
# Appendix: Explicit clock and reset
Chisel modules have a default clock and reset that are implicitly used by every register created inside them.
There are times where you want to be able to override this default behavior; perhaps you have a black box that generates a clock or reset signal, or you have a multi-clock design.

Chisel provides constructs for dealing with these cases.
Clocks and resets can be overridden separately or together with `withClock() {}`, `withReset() {}`, and `withClockAndReset() {}`.
The following code blocks will give examples of using these functions.

One thing to be aware of is that `reset` (as of this tutorial's writing) is always synchronous and of type `Bool`.
Clocks have their own type in Chisel (`Clock`) and should be declared as such.
*`Bool`s can be converted to `Clock`s by calling `asClock()` on them, but you should be careful that you aren't doing something silly.*

Also note that `chisel-testers` do not currently have complete support for multi-clock designs.

<span style="color:blue">**Example: Multi-Clock Module**</span><br>
A module with multiple clocks and reset signals.

In [17]:
// we need to import multi-clock features
import chisel3.experimental.{withClock, withReset, withClockAndReset}

class ClockExamples extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(10.W))
    val alternateReset    = Input(Bool())
    val alternateClock    = Input(Clock())
    val outImplicit       = Output(UInt())
    val outAlternateReset = Output(UInt())
    val outAlternateClock = Output(UInt())
    val outAlternateBoth  = Output(UInt())
  })

  val imp = RegInit(0.U(10.W))
  imp := io.in
  io.outImplicit := imp

  withReset(io.alternateReset) {
    // everything in this scope with have alternateReset as the reset
    val altRst = RegInit(0.U(10.W))
    altRst := io.in
    io.outAlternateReset := altRst
  }

  withClock(io.alternateClock) {
    val altClk = RegInit(0.U(10.W))
    altClk := io.in
    io.outAlternateClock := altClk
  }

  withClockAndReset(io.alternateClock, io.alternateReset) {
    val alt = RegInit(0.U(10.W))
    alt := io.in
    io.outAlternateBoth := alt
  }
}

println(getVerilog(new ClockExamples))

Elaborating design...
Done elaborating.
module ClockExamples(
  input        clock,
  input        reset,
  input  [9:0] io_in,
  input        io_alternateReset,
  input        io_alternateClock,
  output [9:0] io_outImplicit,
  output [9:0] io_outAlternateReset,
  output [9:0] io_outAlternateClock,
  output [9:0] io_outAlternateBoth
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
  reg [31:0] _RAND_1;
  reg [31:0] _RAND_2;
  reg [31:0] _RAND_3;
`endif // RANDOMIZE_REG_INIT
  reg [9:0] imp; // @[cmd16.sc 14:20]
  reg [9:0] REG; // @[cmd16.sc 20:25]
  reg [9:0] REG_1; // @[cmd16.sc 26:25]
  reg [9:0] REG_2; // @[cmd16.sc 32:22]
  assign io_outImplicit = imp; // @[cmd16.sc 16:18]
  assign io_outAlternateReset = REG; // @[cmd16.sc 22:26]
  assign io_outAlternateClock = REG_1; // @[cmd16.sc 28:26]
  assign io_outAlternateBoth = REG_2; // @[cmd16.sc 34:25]
  always @(posedge clock) begin
    if (reset) begin // @[cmd16.sc 14:20]
      imp <= 10'h0; // @[cmd16.sc 14:20]
    end else begin

import chisel3.experimental.{withClock, withReset, withClockAndReset}


defined class ClockExamples

---
# Wrap Up
Great job completing this section!! You've now learned how to create registers and write sequential logic in Chisel, which means you have enough basic building blocks to write real circuits.

The next section will combine everything we've learned into one example! If you need a little more encouragement, just remember these words from an expert Chisel user:

![BobRoss](http://i.qkme.me/3qbd5u.jpg)

---
# You're done!

[Return to the top.](#top)